# Create video index

Code authored by: Shaw Talebi <br>

Video link: https://youtu.be/6qCrvlHRhcM

### imports

In [1]:
import polars as pl
from sentence_transformers import SentenceTransformer

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### load data

In [2]:
df = pl.read_parquet('data/video-transcripts.parquet')
df.head()

video_id,datetime,title,transcript
str,datetime[μs],str,str
"""03x2oYg9oME""",2024-04-25 15:16:00,"""Data Science P…","""this video is …"
"""O5i_mMUM94c""",2024-04-19 14:05:54,"""How I’d learne…","""here's how I'd…"
"""xm9devSQEqU""",2024-04-18 15:59:02,"""4 Skills You N…","""although it is…"
"""Z6CmuVEi7QY""",2024-04-11 10:00:27,"""How I'd Learn …","""when I was fir…"
"""INlCLmWlojY""",2024-04-04 18:45:00,"""I Was Wrong Ab…","""last year I qu…"


### embed titles and transcripts

In [3]:
model_name = 'all-MiniLM-L6-v2'
column_name_list = ['title', 'transcript']

In [4]:
model = SentenceTransformer(model_name)

for column_name in column_name_list:
    # generate embeddings
    embedding_arr = model.encode(df[column_name].to_list())

    # store embeddings in a dataframe
    schema_dict = {column_name+'_embedding-'+str(i): float for i in range(embedding_arr.shape[1])}
    df_embedding = pl.DataFrame(embedding_arr, schema=schema_dict)

    # append embeddings to video index
    df = pl.concat([df, df_embedding], how='horizontal')

In [8]:
df.shape

(83, 772)

In [5]:
df.head()

video_id,datetime,title,transcript,title_embedding-0,title_embedding-1,title_embedding-2,title_embedding-3,title_embedding-4,title_embedding-5,title_embedding-6,title_embedding-7,title_embedding-8,title_embedding-9,title_embedding-10,title_embedding-11,title_embedding-12,title_embedding-13,title_embedding-14,title_embedding-15,title_embedding-16,title_embedding-17,title_embedding-18,title_embedding-19,title_embedding-20,title_embedding-21,title_embedding-22,title_embedding-23,title_embedding-24,title_embedding-25,title_embedding-26,title_embedding-27,title_embedding-28,title_embedding-29,title_embedding-30,title_embedding-31,title_embedding-32,…,transcript_embedding-347,transcript_embedding-348,transcript_embedding-349,transcript_embedding-350,transcript_embedding-351,transcript_embedding-352,transcript_embedding-353,transcript_embedding-354,transcript_embedding-355,transcript_embedding-356,transcript_embedding-357,transcript_embedding-358,transcript_embedding-359,transcript_embedding-360,transcript_embedding-361,transcript_embedding-362,transcript_embedding-363,transcript_embedding-364,transcript_embedding-365,transcript_embedding-366,transcript_embedding-367,transcript_embedding-368,transcript_embedding-369,transcript_embedding-370,transcript_embedding-371,transcript_embedding-372,transcript_embedding-373,transcript_embedding-374,transcript_embedding-375,transcript_embedding-376,transcript_embedding-377,transcript_embedding-378,transcript_embedding-379,transcript_embedding-380,transcript_embedding-381,transcript_embedding-382,transcript_embedding-383
str,datetime[μs],str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""03x2oYg9oME""",2024-04-25 15:16:00,"""Data Science P…","""this video is …",-0.015707,-0.000803,-0.000592,0.014483,0.054659,-0.077496,-0.002334,-0.037766,-0.076303,0.095494,-0.023669,-0.065215,0.079585,-0.008432,-0.049648,0.090498,-0.083202,-0.051613,0.008807,-0.082336,-0.02177,-0.031772,-0.010281,0.043854,-0.042952,0.085398,0.053058,0.056221,-0.006999,0.011987,-0.025156,0.063567,0.051586,…,0.081404,-0.00473,-0.019275,0.015354,0.01367,-0.048947,0.014555,0.058803,-0.07865,0.039052,0.018853,0.030402,0.010164,-0.01578,0.028731,0.039881,0.107271,-0.086652,0.068573,-0.128907,0.000532,-0.000074,-0.030465,-0.041026,0.044691,-0.009989,0.077964,0.073521,0.09619,0.040807,-0.009223,-0.074324,0.063215,0.017714,0.006442,-0.029179,-0.017104
"""O5i_mMUM94c""",2024-04-19 14:05:54,"""How I’d learne…","""here's how I'd…",-0.012019,-0.064174,0.070952,0.070263,0.058338,-0.06399,0.090048,-0.018103,-0.030347,0.045006,0.028399,-0.050822,0.03355,0.021837,-0.054941,0.031815,-0.046915,-0.026955,-0.030183,-0.031991,0.006332,0.03895,0.060244,0.002082,0.026674,0.129466,-0.003821,-0.035118,0.048048,-0.034193,0.00103,0.066846,0.035806,…,0.041583,-0.014911,-0.010122,-0.045581,-0.045081,-0.0292,-0.0444,0.030496,-0.059147,0.060487,-0.057393,-0.016889,0.000606,-0.056332,0.05195,0.050159,0.032203,-0.130664,0.076355,-0.093234,0.004911,0.024305,0.036537,-0.028809,-0.008713,-0.044622,-0.007744,0.113142,0.059153,0.001777,0.033651,-0.106226,0.103233,-0.017869,0.051937,-0.127236,0.036246
"""xm9devSQEqU""",2024-04-18 15:59:02,"""4 Skills You N…","""although it is…",0.018166,-0.090621,-0.012552,0.020742,-0.093658,-0.107941,0.005711,0.009971,-0.104353,0.022222,-0.064549,-0.097381,-0.042711,0.019615,-0.034009,0.100303,-0.061772,0.003508,-0.03378,-0.13128,-0.017863,-0.057626,0.015556,-0.072058,-0.010606,0.056719,0.013839,-0.041414,0.017813,-0.005246,0.029335,-0.02157,0.042233,…,0.031496,-0.040242,0.011707,-0.007683,0.008842,0.012777,-0.005904,0.104284,-0.009653,0.082479,-0.040231,-0.030715,0.048085,0.003187,0.087079,-0.013218,0.069204,-0.050548,0.015828,-0.04139,-0.011192,0.034093,0.049186,0.048458,0.004615,0.032301,0.075

### save index to file

In [6]:
df.write_parquet('data/video-index.parquet')